# Clasificación del Sentimiento Económico

En esta notebook se usa el modelo de clasificación desarrollado anteriormente para clasificar las cartas de los accionistas según su sentimiento económico.

1. En primer lugar se ajusta el modelo desarrollado a los datos.
2. Luego se usa el modelo para clasificar las cartas a los accionistas.

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import unidecode

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

# Carga de los datos

En la siguiente celda se cargan los datos de sentimiento económico. 

Luego:
- Se reemplaza la etiqueta de sentimiento: 2 (neutral), por 0 (no positivo). El clasificador se entrenará sólo para identificar el sentimiento positivo.
- Se eliminaron los *missing values* (NA).
- Se seleccionó sólo un conjunto de las variables de la base de datos: (1) response: dummy igual 1 para los textos con sentimiento positivo, y 0 en otro caso; y (2) texto: noticia o fragmento de carta a los accionistas con una polaridad específica.

In [2]:
# Se eliminan los textos que provinen de memorias y los que tienen respuestas neutrales.
data = pd.read_csv('Sentimientos.csv')
data['response'][data['response'] == 2.0] = 0
data = data.dropna().reset_index(drop=True)
data = data[['response', 'texto']]

# Pre-procesamiento

El procesamiento consiste en simplificar los textos, mediante la función de la siguiente celda, que:
- Pone todas las palabras en minúsculas.
- Quita números y caracteres especiales.
- Separa los textos en palabras individuales.
- Mantiene sólo las palabras que no sean Stopwords (remueve las Stopwords)
- Une las palabras nuevamente para formar un texto.

Luego se vectorizan los textos, de forma de darles una estructura matricial, donde las filas son los textos económicos y las columnas el vocabulario, las 3000 palabras más comunes en el total de textos económicos. Los valores de la matriz están dados por distintas medidas: recuento, porcentaje y tf-idf, para palabras individuales y los ngrams 1 y 2.

In [3]:
# Stopwords
sw = pd.read_csv('spanish.txt', header=None, names=['stopwords'])
stopwords = sw['stopwords'].tolist()

# Función para pre-procesar textos
def preprocess(s):
    s = s.lower()
    s = re.sub('[0-9]+', '', s) 
    s = re.sub('[!"#$%&()*+,-./:;<=>¿?@[\\]^_`{|}~\t—’‘“”]', '', s)
    tokens = nltk.tokenize.word_tokenize(s) 
    tokens = [t for t in tokens if t not in stopwords] 
    tokens = [unidecode.unidecode(t) for t in tokens]
    jtokens = ' '.join(tokens)
    return jtokens

In [4]:
# Preprocesamiento de los textos
pdata = [preprocess(t) for t in data['texto']]

In [5]:
# Vectorización
vect1 = CountVectorizer(max_features=3000)
vdat = vect1.fit_transform(pdata)
data1 = pd.DataFrame(vdat.toarray(), columns=vect1.get_feature_names())

vect2 = CountVectorizer(max_features=3000, ngram_range=(1,2))
vdat = vect2.fit_transform(pdata)
data2 = pd.DataFrame(vdat.toarray(), columns=vect2.get_feature_names())

data1sum = data1.sum(axis=1)
data3 = data1.divide(data1sum, axis=0)

data2sum = data2.sum(axis=1)
data4 = data2.divide(data2sum, axis=0)

vect5 = TfidfVectorizer(max_features=3000)
vdat = vect5.fit_transform(pdata)
data5 = pd.DataFrame(vdat.toarray(), columns=vect5.get_feature_names())

vect6 = TfidfVectorizer(max_features=3000, ngram_range=(1,2))
vdat = vect6.fit_transform(pdata)
data6 = pd.DataFrame(vdat.toarray(), columns=vect6.get_feature_names())

# Modelo de clasificación

A continuación se ajusta el modelo de clasificación a los datos, usando los parámetros óptimos determinados anteriormente.

Recordar que el modelo se compone de:
- Seis regresiones logísticas para las seis bases de datos, que fueron preprocesadas de manera diferente.
- Un metamodelo, una regresión logística, que predice en base a las predicciones de los modelos anteriores.

En esta oportunidad el modelo se ajusta al total de datos y, luego, se optimiza el threshold, que es el umbral sobre el cual una predicción del modelo, que es una probabilidad, se considera como sentimiento positivo (valor 1).

In [6]:
# Separación de la muestra
y = data['response']

In [7]:
# DataFrames vacíos para almacenar los resultados 
predicciones = pd.DataFrame()

In [8]:
# Dataframes con parámetros
parametros_1 = pd.read_csv('parametros_1.csv', index_col=0)
parametros_2 = pd.read_csv('parametros_2.csv', index_col=0)

In [9]:
# Especificación de los modelos
model1 = LogisticRegression(random_state=123, C=parametros_1.loc['data1'][0], penalty=parametros_1.loc['data1'][1])
model2 = LogisticRegression(random_state=123, C=parametros_1.loc['data2'][0], penalty=parametros_1.loc['data2'][1])
model3 = LogisticRegression(random_state=123, C=parametros_1.loc['data3'][0], penalty=parametros_1.loc['data3'][1])
model4 = LogisticRegression(random_state=123, C=parametros_1.loc['data4'][0], penalty=parametros_1.loc['data4'][1])
model5 = LogisticRegression(random_state=123, C=parametros_1.loc['data5'][0], penalty=parametros_1.loc['data5'][1])
model6 = LogisticRegression(random_state=123, C=parametros_1.loc['data6'][0], penalty=parametros_1.loc['data6'][1])

In [10]:
# Especificación de los inputs de la siguiente celda
inputs = [
    ('data1', data1, model1),
    ('data2', data2, model2),
    ('data3', data3, model3),
    ('data4', data4, model4),
    ('data5', data5, model5),
    ('data6', data6, model6)
]

In [11]:
# Ajuste de los modelos de primera línea
for name, data, model in inputs:
    model.fit(data, y)
    predicciones[name] = model.predict_proba(data)[:,1]

In [12]:
# Ajuste del meta modelo
Mmodel = LogisticRegression(random_state=123,
                            C=parametros_2['C'].iloc[0],
                            penalty=parametros_2['penalty'].iloc[0])
Mmodel.fit(predicciones, y)
y_pred_proba = Mmodel.predict_proba(predicciones)[:,1]

In [13]:
# Función para transformar probabilidad en label
def to_labels(y_pred_proba, threshold):
	return (y_pred_proba >= threshold).astype('int')

thresholds = np.linspace(0, 1, 300)

# Optimización del threshold en base a F1 Score
scores = [f1_score(y, to_labels(y_pred_proba, t)) for t in thresholds]
ix = np.argmax(scores)
print('Threshold=%.4f, F-Score=%.4f' % (thresholds[ix], scores[ix]))

Threshold=0.0033, F-Score=1.0000


In [14]:
# Verificación de las predicciones
y_pred = (y_pred_proba >= thresholds[ix]).astype(int)
y_pred = pd.DataFrame({'y_pred': y_pred, 'y_pred_proba': y_pred_proba}, index=y.index)

pred = pd.concat([y, y_pred], axis=1)
pred['Accuracy'] = (pred['response']==pred['y_pred'])
pred.to_csv('preds.csv')

# Clasificación de las cartas de accionistas (párrafos)

En esta sección se clasifican las cartas a los accionistas usando el modelo clasificador ajustado anteriormente.

En primer lugar se cargan los datos, donde las cartas fueron separadas por párrafos. La columna *index* identifica la carta a la que pertenece cada párrafo.

Recordar que de  esta manera, cada párrafo se considera como un texto individual, que luego será clasificado por el algoritmo clasificador de sentimiento económico. 

Esto se hace así por los siguientes motivos: 
- (1) los textos de las memorias son principalmente optimistas, por lo que si se opta por clasificar las cartas completas, todas serían optimistas.
- (2) aunque las cartas sean principalmente optimistas, pueden contener párrafos pesimistas, neutrales y optimistas, por lo que clasificar cada párrafo por separado permite medir el grado de optimismo de cada carta.


Por último, la forma de agregar las clasificaciones por texto (por carta) es obtener el porcentaje de párrafos optimistas sobre el total de párrafos. Así, también se logra un tercer objetivo: 
- (3) normalizar por la extensión de las cartas, ya que algunas cartas son mucho más extensas que otras.


In [15]:
parrafos = pd.read_csv('parrafos.csv')
parrafos.head()

,index,nombre,industria,cargo,carta,autor
0,0,AES Gener,Energía,Presidente del Directorio y Gerente General,La gestión oportuna de los impactos de la pand...,Presidente
1,0,AES Gener,Energía,Presidente del Directorio y Gerente General,Quisiera comenzar destacando que a pesar de qu...,Presidente
2,0,AES Gener,Energía,Presidente del Directorio y Gerente General,En nuestro objetivo de liderar la transformaci...,Presidente
3,0,AES Gener,Energía,Presidente del Directorio y Gerente General,Un hito relevante del Plan de 1.000 MW de proy...,Presidente
4,0,AES Gener,Energía,Presidente del Directorio y Gerente General,Los requerimientos que surgieron de este proce...,Presidente


Luego, se pre procesan las cartas, usando la función descrita anteriormente.

In [16]:
pdata = [preprocess(t) for t in parrafos['carta']]

El siguiente paso es vectorizar, para lo cual se usan los mismos vectorizadores anteriores. Es decir, se usa el vocabulario de los textos económicos, no el de las cartas a los accionistas, porque el modelo fue estimado en base a dicho vocabulario.

In [17]:
# Vectorización
vdat = vect1.transform(pdata)
data1 = pd.DataFrame(vdat.toarray(), columns=vect1.get_feature_names())

vdat = vect2.transform(pdata)
data2 = pd.DataFrame(vdat.toarray(), columns=vect2.get_feature_names())

data1sum = data1.sum(axis=1)
data3 = data1.divide(data1sum, axis=0)

data2sum = data2.sum(axis=1)
data4 = data2.divide(data2sum, axis=0)

vdat = vect5.transform(pdata)
data5 = pd.DataFrame(vdat.toarray(), columns=vect5.get_feature_names())

vdat = vect6.transform(pdata)
data6 = pd.DataFrame(vdat.toarray(), columns=vect6.get_feature_names())

In [18]:
# Especificación de los inputs de la siguiente celda
inputs = [
    ('data1', data1, model1),
    ('data2', data2, model2),
    ('data3', data3, model3),
    ('data4', data4, model4),
    ('data5', data5, model5),
    ('data6', data6, model6)
]

A continuación se hacen las predicciones de sentimiento para los párrafos de las cartas a los accionistas, usando el modelo clasificador.

In [19]:
# Predicciones de los modelos de primera línea
predicciones = pd.DataFrame()

for name, data, model in inputs:
    predicciones[name] = model.predict_proba(data)[:,1]

In [20]:
# Predicciones del meta modelo
y_pred_proba = Mmodel.predict_proba(predicciones)[:,1]

Esta última celda clasifica los párrafos según el threshold encontrado anteriormente.

In [21]:
# Predicciones
y_pred = (y_pred_proba >= thresholds[ix]).astype(int)
y_pred = pd.DataFrame({'y_pred': y_pred, 'y_pred_proba': y_pred_proba}, index=parrafos.index)

pred = pd.concat([parrafos, y_pred], axis=1)
pred.to_csv('class_parrafos.csv', index=False)

# Clasificación de las cartas de accionistas (cartas completas)

A continuación se repite el procedimiento anterior pero se clasifican las cartas completas, sin separar por párrafo. Como se esperaba, se observa que la gran mayoría de las cartas son clasificadas como **optimistas**, lo que sugiere que, para obtener más precisión en la medición del sentimiento económico, es recomendable optar por clasificar cada párrafo por separado, y luego agregar las clasificaciones de los párrafos para cada carta.

In [22]:
cartas = pd.read_csv('Cartas a los accionistas.csv')
cartas = cartas.dropna().reset_index(drop=True)
print(cartas.shape)
cartas.head()

(41, 7)


,nemotecnico,razon social,nombre,industria,autor,cargo,carta
0,AESGENER,AES GENER S.A.,AES Gener,Energía,Julián Nebreda y Ricardo Falú,Presidente del Directorio y Gerente General,La gestión oportuna de los impactos de la pand...
1,AGUAS-A,"AGUAS ANDINAS S.A., SERIE A",Aguas Andinas,Servicios Básicos,Claudio Muñoz,Presidente del Directorio,"Como cada año, tengo el agrado de presentarles..."
2,AGUAS-A,"AGUAS ANDINAS S.A., SERIE A",Aguas Andinas,Servicios Básicos,Marta Colet,Gerente General,"""Sin duda, la pandemia por Covid-19 marcó nues..."
3,CHILE,BANCO DE CHILE,Banco de Chile,Financiera,Pablo Granifo,Presidente del Directorio,Con gran orgullo y satisfacción me dirijo a us...
4,CHILE,BANCO DE CHILE,Banco de Chile,Financiera,Eduardo Ebensperger,Gerente General,"""Me siento muy honrado de compartir con ustede..."


In [23]:
pdata = [preprocess(t) for t in cartas['carta']]

In [24]:
# Vectorización
vdat = vect1.transform(pdata)
data1 = pd.DataFrame(vdat.toarray(), columns=vect1.get_feature_names())

vdat = vect2.transform(pdata)
data2 = pd.DataFrame(vdat.toarray(), columns=vect2.get_feature_names())

data1sum = data1.sum(axis=1)
data3 = data1.divide(data1sum, axis=0)

data2sum = data2.sum(axis=1)
data4 = data2.divide(data2sum, axis=0)

vdat = vect5.transform(pdata)
data5 = pd.DataFrame(vdat.toarray(), columns=vect5.get_feature_names())

vdat = vect6.transform(pdata)
data6 = pd.DataFrame(vdat.toarray(), columns=vect6.get_feature_names())

In [25]:
# Especificación de los inputs de la siguiente celda
inputs = [
    ('data1', data1, model1),
    ('data2', data2, model2),
    ('data3', data3, model3),
    ('data4', data4, model4),
    ('data5', data5, model5),
    ('data6', data6, model6)
]

In [26]:
# Predicciones de los modelos de primera línea
predicciones = pd.DataFrame()

for name, data, model in inputs:
    predicciones[name] = model.predict_proba(data)[:,1]

In [27]:
# Predicciones del meta modelo
y_pred_proba = Mmodel.predict_proba(predicciones)[:,1]

In [28]:
# Predicciones
y_pred = (y_pred_proba >= thresholds[ix]).astype(int)
y_pred = pd.DataFrame({'y_pred': y_pred, 'y_pred_proba': y_pred_proba}, index=cartas.index)

pred = pd.concat([cartas, y_pred], axis=1)
pred.to_csv('class_cartas.csv', index=False)